In [7]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [8]:
#!pip install catboost

In [243]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [244]:
myfolder = "./"

# **Columns' names**

In [245]:
#Columns' names
'''
1)  unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  21
'''
unitNames = ['UnitNumber']
timeCycles = ["TimeInCycles"]
operSets = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes = ["SensorMes"+str(j) for j in range(1, 22)] # de 1 à 21
columnsNames = unitNames + timeCycles + operSets +sensorMes

# **Datasets loading**

In [246]:
def data_loading(x):
  train_path = myfolder + "train_"+ x +".txt"
  test_path = myfolder + "test_"+ x +".txt"
  rul_path = myfolder + "RUL_"+ x +".txt"
  train = pd.read_csv(train_path, delim_whitespace=True, names=columnsNames)
  test = pd.read_csv(test_path, delim_whitespace=True, names=columnsNames)
  rul = pd.read_csv(rul_path, delim_whitespace=True, names=["RUL_FD"])
  return train, test, rul

train_fd001, test_fd001, rul_fd001 = data_loading("FD001")
train_fd002, test_fd002, rul_fd002 = data_loading("FD002")
train_fd003, test_fd003, rul_fd003 = data_loading("FD003")
train_fd004, test_fd004, rul_fd004 = data_loading("FD004")

In [247]:
# Forcer l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213


# **RUL column generation for train and test set**

In [248]:
def rul_train_generation(x):
  rul = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  rul.columns = ['UnitNumber', 'max']
  x = x.merge(rul, on=['UnitNumber'], how='left')
  x['RUL'] = x['max'] - x['TimeInCycles']
  x.drop('max', axis=1, inplace=True)
  return x

train_fd001 = rul_train_generation(train_fd001)
train_fd002 = rul_train_generation(train_fd002)
train_fd003 = rul_train_generation(train_fd003)
train_fd004 = rul_train_generation(train_fd004)

In [249]:
train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213,318


In [250]:
def rul_test_generation(x, rul):

  rul["UnitNumber"] = rul.index + 1 # +1 pour que UnitNumber demarre de 1 au lieu de 0, car il s'agit du numero des moteur

  x = x.merge(rul, on=['UnitNumber'], how='left')

  max_cycle = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  max_cycle.columns = ['UnitNumber', 'max']
  x = x.merge(max_cycle, on=['UnitNumber'], how='left')
  x['RUL'] = x['RUL_FD'] + x['max'] - x['TimeInCycles']
  x.drop(['max', 'RUL_FD'], axis=1, inplace=True)

  return x

test_fd001 = rul_test_generation(test_fd001, rul_fd001)
test_fd002 = rul_test_generation(test_fd002, rul_fd002)
test_fd003 = rul_test_generation(test_fd003, rul_fd003)
test_fd004 = rul_test_generation(test_fd004, rul_fd004)

In [251]:
test_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,20.0072,0.700,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,332.52,2323.67,8704.98,1.07,43.83,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,251
1,1,2,24.9984,0.620,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,174.46,1915.21,7999.94,0.93,36.11,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,250
2,1,3,42.0000,0.842,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,137.34,2211.93,8305.38,1.01,41.52,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,249


In [252]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". RUL: ", rul_fd001.shape)
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". RUL: ", rul_fd002.shape)
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". RUL: ", rul_fd003.shape)
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". RUL: ", rul_fd004.shape)

train_FD001:  (20631, 27) . test_FD001:  (13096, 27) . RUL:  (100, 2)
train_FD002:  (53759, 27) . test_FD002:  (33991, 27) . RUL:  (259, 2)
train_FD003:  (24720, 27) . test_FD003:  (16596, 27) . RUL:  (100, 2)
train_FD004:  (61249, 27) . test_FD004:  (41214, 27) . RUL:  (248, 2)


In [253]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". nbr moteurs", len(train_fd001['UnitNumber'].unique()))
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". nbr moteurs", len(train_fd002['UnitNumber'].unique()))
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". nbr moteurs", len(train_fd003['UnitNumber'].unique()))
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". nbr moteurs", len(train_fd004['UnitNumber'].unique()))

train_FD001:  (20631, 27) . test_FD001:  (13096, 27) . nbr moteurs 100
train_FD002:  (53759, 27) . test_FD002:  (33991, 27) . nbr moteurs 260
train_FD003:  (24720, 27) . test_FD003:  (16596, 27) . nbr moteurs 100
train_FD004:  (61249, 27) . test_FD004:  (41214, 27) . nbr moteurs 249


# **All datasets concatenation**

In [254]:
all_data = pd.concat([train_fd001, test_fd001, train_fd002, test_fd002, train_fd003, test_fd003, train_fd004, test_fd004], ignore_index=True)
all_data

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.30,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.30,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.30,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.30,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.30,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265251,248,277,41.9991,0.8401,100.0,445.00,550.30,1364.40,1129.17,3.91,5.72,138.34,2212.35,8351.73,1.02,42.30,130.87,2388.50,8112.61,9.4427,0.02,331,2212,100.0,10.53,6.2620,30
265252,248,278,20.0026,0.7005,100.0,491.19,608.00,1494.75,1260.88,9.35,13.66,334.75,2324.23,8758.69,1.07,44.53,314.51,2388.33,8086.83,9.2772,0.02,366,2324,100.0,24.33,14.6486,29
265253,248,279,34.9988,0.8413,100.0,449.44,555.92,1370.65,1130.97,5.48,8.00,194.92,2223.57,8370.49,1.02,42.33,182.76,2388.64,8100.84,9.3982,0.02,336,2223,100.0,14.69,8.8389,28
265254,248,280,20.0027,0.7000,100.0,491.19,608.19,1489.11,1256.25,9.35,13.66,333.15,2324.19,8753.04,1.07,44.78,314.17,2388.37,8085.24,9.2727,0.03,366,2324,100.0,24.44,14.6887,27


In [255]:
def unitnumbre_train80_test20(df, sample_size):
    # Initialisation des DataFrames d'entraînement et de test
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()

    # Liste unique des valeurs de 'UnitNumber'
    unit_numbers = df['UnitNumber'].unique()

    np.random.seed(42) # pour la reproductibilité

    # Pour chaque 'UnitNumber', diviser les données en ensembles d'entraînement et de test
    for unit in unit_numbers:
        unit_df = df[df['UnitNumber'] == unit]
        
        # Mélanger l'index du DataFrame
        shuffled_df = unit_df.sample(frac=1, random_state=42)
        
        # Prendre les 'sample_size' premières lignes pour l'entraînement et le reste pour le test
        train_size = int((sample_size * len(shuffled_df))/100)
        
        unit_train = shuffled_df[:train_size]
        unit_test = shuffled_df[train_size:]
        
        train_df = pd.concat([train_df, unit_train])
        test_df = pd.concat([test_df, unit_test])
    
    return train_df, test_df

train_fd, test_fd = split_df(all_data, sample_size=80)

train_fd.reset_index(drop=True, inplace=True)
test_fd.reset_index(drop=True, inplace=True)

In [256]:
#cette foncetion fait la meme chose que la precedente

'''from sklearn.model_selection import train_test_split

def data_split2(df):
    # Initialisation des DataFrames d'entraînement et de test
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()

    # Liste unique des valeurs de 'UnitNumber'
    unit_numbers = df['UnitNumber'].unique()

    # Pour chaque 'UnitNumber', diviser les données en ensembles d'entraînement et de test
    for unit in unit_numbers:
        unit_df = df[df['UnitNumber'] == unit]
        
        unit_train, unit_test = train_test_split(unit_df, test_size=0.2, random_state=42) # Vous pouvez changer la graine aléatoire si vous voulez

        train_df = pd.concat([train_df, unit_train])
        test_df = pd.concat([test_df, unit_test])
    return train_df, test_df
train_df1, test_df1 = data_split2(all_data)
train_df1.reset_index(drop=True, inplace=True)
test_df1.reset_index(drop=True, inplace=True)'''

"from sklearn.model_selection import train_test_split\n\ndef data_split2(df):\n    # Initialisation des DataFrames d'entraînement et de test\n    train_df = pd.DataFrame()\n    test_df = pd.DataFrame()\n\n    # Liste unique des valeurs de 'UnitNumber'\n    unit_numbers = df['UnitNumber'].unique()\n\n    # Pour chaque 'UnitNumber', diviser les données en ensembles d'entraînement et de test\n    for unit in unit_numbers:\n        unit_df = df[df['UnitNumber'] == unit]\n        \n        unit_train, unit_test = train_test_split(unit_df, test_size=0.2, random_state=42) # Vous pouvez changer la graine aléatoire si vous voulez\n\n        train_df = pd.concat([train_df, unit_train])\n        test_df = pd.concat([test_df, unit_test])\n    return train_df, test_df\ntrain_df1, test_df1 = data_split2(all_data)\ntrain_df1.reset_index(drop=True, inplace=True)\ntest_df1.reset_index(drop=True, inplace=True)"

# **Data normalization**

In [257]:
def normalised_df(train, test):

  from sklearn.preprocessing import MinMaxScaler

  # Instancier l'objet MinMaxScaler pour normaliser les données
  scaler = MinMaxScaler()

  # Normaliser train
  train_scaled = scaler.fit_transform(train)
  train_df = pd.DataFrame(train_scaled)

  # Normaliser test
  test_scaled = scaler.fit_transform(test)
  test_df = pd.DataFrame(test_scaled)


  train_df.columns = train.columns
  test_df.columns = test.columns
    
  return train_df, test_df

# **Data splitting**

In [258]:
def data_split(train, test):

  # data split
  X_train = train.drop('RUL', axis=1)
  Y_train = train['RUL']
  X_test = test.drop('RUL', axis=1)
  Y_test = test['RUL']
    
  return X_train, Y_train, X_test, Y_test

# **The KNN Regressor model**

In [264]:
def my_knn_regressor(train, test):

    X_train, Y_train, X_test, Y_test = data_split(train, test)

    # Créer le modèle KNN Regressor
    model = KNeighborsRegressor()

    # Définir les paramètres à tester dans la recherche par grille
    param_grid = {
      'n_neighbors': np.arange(1, 100),
      'weights': ['uniform', 'distance'],
      'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],
      }

    # Créer l'objet GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

    # Effectuer la recherche par grille sur les données d'entraînement
    grid_search.fit(X_train, Y_train)

    # Afficher les meilleurs paramètres trouvés
    print("Meilleurs paramètres trouvés :")
    print(grid_search.best_params_)

    '''#-------------Train---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
    y_pred_train = grid_search.predict(X_train)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_train = mean_squared_error(Y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(Y_train, y_pred_train)
    mape_train = np.mean(np.abs((Y_train - y_pred_train) / Y_train)) * 100

    print('\n=============================Train=============================')
    print('MSE : ',mse_train * 100,'%')
    print('RMSE : ',rmse_train * 100,'%')
    print('MAE : ',mae_train * 100,'%')
    print('MAPE : ',mape_train,'%')'''

    #-------------Test---------------------
    # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
    y_pred_test = grid_search.predict(X_test)
    # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
    mse_test = mean_squared_error(Y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(Y_test, y_pred_test)
    mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100

    print('\n=============================Test=============================')
    print('MSE : ',mse_test * 100,'%')
    print('RMSE : ',rmse_test * 100,'%')
    print('MAE : ',mae_test * 100,'%')
    print('MAPE : ',mape_test,'%')

In [ ]:
my_knn_regressor(train_fd, test_fd)